<a href="https://colab.research.google.com/github/gyeongdong13/Competition/blob/main/%ED%86%B5%ED%95%A9%EB%B3%B8%EC%84%A0(XGB%2C_LGBM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/시니어/통합본선 최종 - 이상치 변환.csv', encoding='utf-8')

df.head()

,행정동,계총인구수,요보호 노인인구 비율,평균소득,총가구수,영역둘레(평균상권),영역면적(평균상권),의료기관수,1의료기관 당 요보호 노인인구 (종속변수),행정동 면적,...,보건소,보건지소,보건진료소,상급종합,의원,종합병원,치과병원,치과의원,한방병원,한의원
0,가음정동,38875.0,2.22,4727.67,7924,657.13,35157.54,37.0,0.06,2.89,...,0,0,0,0,15.0,0,0,12.0,0,9.0
1,가포동,2279.0,3.95,3001.30,15529,10229.23,2705034.00,1.0,3.95,4.47,...,0,0,0,0,0.0,0,0,0.0,0,0.0
2,경화동,10824.0,6.23,2241.70,3198,1313.01,153242.20,16.0,0.39,2.12,...,0,0,0,0,6.0,0,0,3.0,0,7.0
3,교방동,15947.0,7.73,2323.80,19918,1008.87,108515.20,10.0,0.77,2.73,...,0,0,0,0,3.0,0,0,4.0,0,3.0
4,구산면,4072.0,16.90,2139.00,14839,12028.37,4010639.00,3.0,5.63,43.64,...,0,1,1,0,1.0,0,0,0.0,0,0.0


In [ ]:
df = df.iloc[:,1:]
df.head()

,계총인구수,요보호 노인인구 비율,평균소득,총가구수,영역둘레(평균상권),영역면적(평균상권),의료기관수,1의료기관 당 요보호 노인인구 (종속변수),행정동 면적,상권 개수,...,보건소,보건지소,보건진료소,상급종합,의원,종합병원,치과병원,치과의원,한방병원,한의원
0,38875.0,2.22,4727.67,7924,657.13,35157.54,37.0,0.06,2.89,83,...,0,0,0,0,15.0,0,0,12.0,0,9.0
1,2279.0,3.95,3001.30,15529,10229.23,2705034.00,1.0,3.95,4.47,2,...,0,0,0,0,0.0,0,0,0.0,0,0.0
2,10824.0,6.23,2241.70,3198,1313.01,153242.20,16.0,0.39,2.12,20,...,0,0,0,0,6.0,0,0,3.0,0,7.0
3,15947.0,7.73,2323.80,19918,1008.87,108515.20,10.0,0.77,2.73,21,...,0,0,0,0,3.0,0,0,4.0,0,3.0
4,4072.0,16.90,2139.00,14839,12028.37,4010639.00,3.0,5.63,43.64,11,...,0,1,1,0,1.0,0,0,0.0,0,0.0


In [ ]:
x = df.iloc[:, [1,5,13,14]]
y = df.iloc[:, 7]
x.head(5), y.head(5)

(   요보호 노인인구 비율  영역면적(평균상권)  행정동별 병원간의 평균거리  병원에서 가장 가까운 버스정류장 평균 거리
 0         2.22    35157.54            0.56                     0.20
 1         3.95  2705034.00            0.00                     0.09
 2         6.23   153242.20            0.35                     0.09
 3         7.73   108515.20            0.56                     0.11
 4        16.90  4010639.00            2.47                     0.10,
 0    0.06
 1    3.95
 2    0.39
 3    0.77
 4    5.63
 Name: 1의료기관 당 요보호 노인인구 (종속변수), dtype: float64)

In [ ]:
from sklearn.preprocessing  import StandardScaler

scaler = StandardScaler()
xs = scaler.fit_transform(x) # 각 독립변수의 평균과 표준편차 이용하여 단위 축소
x = pd.DataFrame(xs, columns=x.columns)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

# XGBoost 모델 생성
xgb_model = XGBRegressor(random_state=42)

# LightGBM 모델 생성
lgbm_model = LGBMRegressor(random_state=42)

# XGBoost 모델에 대한 하이퍼파라미터 그리드 설정
xgb_param_grid = {
    'n_estimators': [50, 100, 150],       # 결정 트리의 개수
    'learning_rate': [0.05, 0.1, 0.2],    # 학습률
    'max_depth': [3, 5, 7],               # 트리의 최대 깊이
    'min_child_weight': [1, 5, 10],       # 리프 노드에 필요한 최소 가중치 합
    'subsample': [0.8, 1.0],              # 샘플링 비율
}

# LightGBM 모델에 대한 하이퍼파라미터 그리드 설정
lgbm_param_grid = {
    'n_estimators': [50, 100, 150],       # 결정 트리의 개수
    'learning_rate': [0.05, 0.1, 0.2],    # 학습률
    'max_depth': [3, 5, 7],               # 트리의 최대 깊이
    'min_child_samples': [10, 20, 30],    # 리프 노드에 필요한 최소 샘플 수
    'subsample': [0.8, 1.0],              # 샘플링 비율
}

# GridSearchCV를 사용하여 XGBoost 모델의 최적 하이퍼파라미터 찾기
xgb_grid_search = GridSearchCV(xgb_model, param_grid=xgb_param_grid, cv=5, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)

# GridSearchCV를 사용하여 LightGBM 모델의 최적 하이퍼파라미터 찾기
lgbm_grid_search = GridSearchCV(lgbm_model, param_grid=lgbm_param_grid, cv=5, n_jobs=-1)
lgbm_grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 값 출력
print("XGBoost Best Params:", xgb_grid_search.best_params_)
print("LightGBM Best Params:", lgbm_grid_search.best_params_)

XGBoost Best Params: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 50, 'subsample': 0.8}
LightGBM Best Params: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_samples': 10, 'n_estimators': 50, 'subsample': 0.8}


In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

xgb_model = XGBRegressor(**xgb_grid_search.best_params_, random_state=42)

lgbm_boosting_model = LGBMRegressor(**lgbm_grid_search.best_params_, random_state=42)

# XGB 모델 학습
xgb_model.fit(X_train, y_train)

# LGBM 모델 학습
lgbm_boosting_model.fit(X_train, y_train)

# 테스트 데이터셋을 이용하여 모델을 평가
y_pred_xgb = xgb_model.predict(X_test)
y_pred_lgbm = lgbm_boosting_model.predict(X_test)

# 성능 평가
print("XGB R2 Score:", r2_score(y_test, y_pred_xgb))
print("LGBM R2 Score:", r2_score(y_test, y_pred_lgbm))

print("XGB RMSE : ", mean_squared_error(y_test, y_pred_xgb)**(1/2), "LGBM RMSE : ", mean_squared_error(y_test, y_pred_lgbm)**(1/2))

XGB R2 Score: 0.06389348873040834
LGBM R2 Score: 0.21159351972739715
XGB RMSE :  1.675877085170672 LGBM RMSE :  1.537993933248283
